In [ ]:
%pip install ultralytics
ß
from ultralytics import YOLO

In [ ]:
# Load the models
modelv5_tuning = YOLO("yolov5nu.pt")   # these first three models are for hyperparameter tuning
modelv8_tuning = YOLO("yolov8n.pt")
modelv11_tuning = YOLO("yolo11n.pt")

# Tuning

In [ ]:
import random

search_space = {
    "lr0": (1e-5,1e-4,1e-3,1e-2,1e-1),
    "batch": [4,6,8,10,12,14,16,18,20]
}

#Save the hyperparameter values used in the tuning
lr0_dict = {}
batch_dict = {}

for i in range(15):
  lr0 = search_space["lr0"][random.randint(0,4)]
  batch = search_space["batch"][random.randint(0,8)]

  lr0_dict[i] = lr0
  batch_dict[i] = batch

  modelv5_tuning.train(
    data='wisard_dataset_rgb.yaml', #state whether it's RGB or THERMAL 
    epochs=5,
    imgsz=640,
    project='runs/v5_RGB', # change runs/VERSION_MODALITY
    batch=batch,
    lr0=lr0,
    optimizer = 'SGD',
    device='mps', # mpu is for MAC, change to 'auto' or remove this if you're using Windows
    workers=4
  )


# Tuning Results

In [ ]:
import pandas as pd

result_paths = ["train4","train42","train422","train4222","train42222"] #state all output folders from training done during tuning

results_df = pd.DataFrame()

for i, path in enumerate(result_paths): #accesses the results.csv file from each output folders
  # Path to the results.csv file from tuning
  results_path = f'runs/v5_RGB/{path}/results.csv' #change the folder version and modality according to what you've tuned

  # Load results
  df = pd.read_csv(results_path)

  #Obtain the saved hyperparameter values during tuning process
  df['lr0'] = lr0_dict[i]
  df['batch'] = batch_dict[i]

  #Concat to results_df
  df['run_name'] = path  # Track which run it came from
  results_df = pd.concat([results_df, df], ignore_index=True)


# Sort by recall (metrics/recall(B))
results_df_sorted = results_df.sort_values(by='metrics/recall(B)', ascending=False) #changed metrics to recall

In [ ]:
results_df_sorted.head(10)